In [ ]:

!pip -q install lightgbm==4.3.0 pyarrow tqdm


from google.colab import drive
import os, gc, warnings, numpy as np, pyarrow.parquet as pq, lightgbm as lgb, tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

BASE_PATH   = "/content/drive/My Drive/"            # GDrive root
FILE        = "IC00_20230101_20241231.parquet"      # Parquet file name

warnings.filterwarnings("ignore")
drive.mount('/content/drive', force_remount=True)
pf   = pq.ParquetFile(os.path.join(BASE_PATH, FILE))
cols = pf.schema.names


max_lvl = max(i for i in range(1, 11) if f"Buy{i}Price" in cols)
print(f"盘口最大档位: {max_lvl} 档")

# 读取完整Parquet文件
table   = pf.read()
full_df = table.to_pandas()

# 取消Pandas显示限制
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(full_df.head(10))




Mounted at /content/drive
盘口最大档位: 5 档
     MDDate     MDTime  SecurityType SecuritySubType SecurityID  \
0  20230103  092900200             8           08001       IC00   
1  20230103  093000200             8           08001       IC00   
2  20230103  093000700             8           08001       IC00   
3  20230103  093001200             8           08001       IC00   
4  20230103  093001700             8           08001       IC00   
5  20230103  093002200             8           08001       IC00   
6  20230103  093002700             8           08001       IC00   
7  20230103  093003200             8           08001       IC00   
8  20230103  093003700             8           08001       IC00   
9  20230103  093004200             8           08001       IC00   

   SecurityIDSource Symbol TradingPhaseCode  PreClosePx  NumTrades  \
0               301   IC00                3      5871.0        NaN   
1               301   IC00                3      5871.0        NaN   
2             

In [8]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # 将MDTime转换为毫秒时间戳（从当日0点开始的总毫秒数）
# def mdtime_to_ms(mdtime):
#     s = str(int(mdtime)).zfill(9)
#     hours = int(s[:2])
#     minutes = int(s[2:4])
#     seconds = int(s[4:6])
#     millis = int(s[6:9])
#     return (hours * 3600 + minutes * 60 + seconds) * 1000 + millis

# # 应用转换函数
# full_df['time_ms'] = full_df['MDTime'].apply(mdtime_to_ms)

# # 定义连续竞价时间段（毫秒表示）
# MORNING_START = 9 * 3600 * 1000 + 30 * 60 * 1000  # 09:30:00.000
# MORNING_END = 11 * 3600 * 1000 + 30 * 60 * 1000   # 11:30:00.000
# AFTERNOON_START = 13 * 3600 * 1000                 # 13:00:00.000
# AFTERNOON_END = 14 * 3600 * 1000 + 57 * 60 * 1000  # 14:57:00.000

# # # 筛选连续竞价数据（排除集合竞价和收盘集合竞价）
# # continuous_mask = (
# #     ((full_df['time_ms'] >= MORNING_START) & (full_df['time_ms'] <= MORNING_END)) |
# #     ((full_df['time_ms'] >= AFTERNOON_START) & (full_df['time_ms'] < AFTERNOON_END))

# # continuous_df = full_df[continuous_mask].copy()

# # 按日期排序
# continuous_df = continuous_df.sort_values(['MDDate', 'time_ms'])

# # 计算时间间隔（毫秒）
# continuous_df['prev_time_ms'] = continuous_df.groupby('MDDate')['time_ms'].shift(1)
# continuous_df['time_diff_ms'] = continuous_df['time_ms'] - continuous_df['prev_time_ms']

# # 移除无效行（首行和跨天行）
# continuous_df = continuous_df.dropna(subset=['time_diff_ms'])
# continuous_df = continuous_df[continuous_df['time_diff_ms'] > 0]  # 移除负间隔（跨天）

# # 统计间隔分布
# interval_stats = continuous_df['time_diff_ms'].value_counts().sort_index()
# total_intervals = len(continuous_df)
# non_500ms = continuous_df[continuous_df['time_diff_ms'] != 500]
# non_500ms_ratio = len(non_500ms) / total_intervals * 100

# print(f"【连续竞价阶段统计】")
# print(f"总记录数: {len(full_df):,} | 连续竞价记录数: {len(continuous_df):,}")
# print(f"500ms间隔占比: {100 - non_500ms_ratio:.2f}%")
# print(f"非500ms间隔占比: {non_500ms_ratio:.2f}%")
# print("\n间隔分布（毫秒）:")
# print(interval_stats.head(10).to_string())


# # 输出异常间隔样例
# if not non_500ms.empty:
#     print("\n【非500ms间隔样例】")
#     sample = non_500ms[['MDDate', 'MDTime', 'time_diff_ms']].head(10).copy()

#     # 添加可读时间格式
#     sample['time_str'] = sample['MDTime'].apply(
#         lambda x: f"{str(x).zfill(9)[:2]}:{str(x).zfill(9)[2:4]}:{str(x).zfill(9)[4:6]}.{str(x).zfill(9)[6:]}")

#     print(sample[['MDDate', 'time_str', 'time_diff_ms']].to_string(index=False))

In [3]:


!pip -q install lightgbm==4.3.0 pyarrow tqdm

import os, gc, warnings, numpy as np, pyarrow.parquet as pq, lightgbm as lgb, tqdm
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

warnings.filterwarnings("ignore")

GDRIVE_ROOT   = "/content/drive/My Drive/"
PARQUET_FILE  = "IC00_20230101_20241231.parquet"
WINDOW_SIZE   = 1_000        # rows to look back (t‑999 … t)
HORIZON       = 500          # rows ahead to predict (t+500)
STRIDE        = 250          # step for selecting t (1 ⇒ every row)
THRESH        = 0.002        # label threshold (relative Δ)
SEED          = 42
VAL_SPLIT     = 0.10
N_ESTIM       = 2_000
LR            = 0.05
EARLY_STOP    = 200
np.random.seed(SEED)


print("▶ Mounting Google Drive …")
drive.mount("/content/drive", force_remount=True)

print(" 阅读中 …")
pf   = pq.ParquetFile(os.path.join(GDRIVE_ROOT, PARQUET_FILE))
cols = pf.schema.names
DEPTH = max(i for i in range(1, 11) if f"Buy{i}Price" in cols)

buy_p_cols  = [f"Buy{i}Price"     for i in range(1, DEPTH + 1)]
sell_p_cols = [f"Sell{i}Price"    for i in range(1, DEPTH + 1)]
buy_q_cols  = [f"Buy{i}OrderQty"  for i in range(1, DEPTH + 1)]
sell_q_cols = [f"Sell{i}OrderQty" for i in range(1, DEPTH + 1)]
extra_cols  = ["Buy1Price", "Sell1Price"]

arrow_tbl = pf.read(columns=extra_cols + buy_p_cols + sell_p_cols + buy_q_cols + sell_q_cols)
N_ROWS     = arrow_tbl.num_rows
print(f"   rows: {N_ROWS}   depth: {DEPTH}")

# 用float32
_to_np = lambda c: arrow_tbl.column(c).to_numpy(zero_copy_only=False).astype(np.float32)

buy_p_np  = np.column_stack([_to_np(c) for c in buy_p_cols ])
sell_p_np = np.column_stack([_to_np(c) for c in sell_p_cols])
buy_q_np  = np.column_stack([_to_np(c) for c in buy_q_cols ])
sell_q_np = np.column_stack([_to_np(c) for c in sell_q_cols])
buy1_np   = _to_np("Buy1Price")
sell1_np  = _to_np("Sell1Price")

# 内存删除
del arrow_tbl; gc.collect()

# --------------------  生出y，并储存在 labels array里--------------------
mid_price = (buy1_np + sell1_np) * 0.5

first_t   = WINDOW_SIZE - 1
last_t    = N_ROWS - HORIZON - 1
all_t     = np.arange(first_t, last_t + 1, STRIDE, dtype=np.int32)

delta_mid = (mid_price[all_t + HORIZON] - mid_price[all_t]) / mid_price[all_t]
labels    = np.zeros_like(delta_mid, np.int8)
labels[delta_mid >  THRESH] = 2
labels[delta_mid < -THRESH] = 0
labels[np.abs(delta_mid) <= THRESH] = 1
print("   labels computed:", len(labels))

# -------------------- 高频因子（根据研报） --------------------

def extract_feat(t: int) -> np.ndarray:
    s = t - WINDOW_SIZE + 1
    ask_p = sell_p_np[s:t+1]
    bid_p = buy_p_np [s:t+1]
    ask_q = sell_q_np[s:t+1]
    # 表示从 s 到 t（包括 t）。
    bid_q = buy_q_np [s:t+1]
    ask_last, bid_last = ask_p[-1], bid_p[-1]

    feat = []
    # v1 snapshot (prices & sizes at t)
    feat.extend(ask_last)
    feat.extend(bid_last)
    feat.extend(ask_q[-1])
    feat.extend(bid_q[-1])
    # v2 spread & mid at t
    feat.extend(ask_last - bid_last)
    feat.extend((ask_last + bid_last) * 0.5)
    # v3 average |Δ| over window
    feat.extend(np.abs(np.diff(ask_p, axis=0)).mean(axis=0))
    feat.extend(np.abs(np.diff(bid_p, axis=0)).mean(axis=0))
    # v4 window means
    feat.append(ask_p.mean())
    feat.append(bid_p.mean())
    feat.append(ask_q.mean())
    feat.append(bid_q.mean())
    # v5 imbalance sums
    feat.append((ask_p - bid_p).sum())
    feat.append((ask_q - bid_q).sum())
    # v6 slopes
    feat.extend((ask_last - ask_p[0]) / (WINDOW_SIZE - 1))
    feat.extend((bid_last - bid_p[0]) / (WINDOW_SIZE - 1))
    feat.extend((ask_q[-1] - ask_q[0]) / (WINDOW_SIZE - 1))
    feat.extend((bid_q[-1] - bid_q[0]) / (WINDOW_SIZE - 1))
    return np.asarray(feat, np.float32)

print("▶ Extracting features …")
X_all = np.stack([extract_feat(t) for t in tqdm.tqdm(all_t)])
print("   X:", X_all.shape, " y:", labels.shape)

# # -------------------- 训练测试分割 --------------------
# num_total = len(all_t)
# train_end = int(num_total * 0.8)
# val_end   = int(num_total * 0.9)

# X_train, y_train = X_all[:train_end], labels[:train_end]
# X_val,   y_val   = X_all[train_end:val_end], labels[train_end:val_end]
# X_test,  y_test  = X_all[val_end:], labels[val_end:]
# print(f"split: Train {len(X_train)} | Val {len(X_val)} | Test {len(X_test)}")


# --------------------  (训练测试) --------------------
SAFE_GAP = 1_000          # 可以改成 0、HORIZON 或 WINDOW_SIZE+HORIZON-1，目前先设置成1000

num_total = len(all_t)

# —— 计算起止索引—————————
train_start_idx = 0
train_end_idx   = int(num_total * 0.8) - 1

valid_start_idx = train_end_idx + SAFE_GAP + 1
valid_end_idx   = int(num_total * 0.9) - 1

test_start_idx  = valid_end_idx + SAFE_GAP + 1
# -------------------------------------------------------------

# —— 实际切片——————————
X_train, y_train = X_all[train_start_idx:train_end_idx+1],          labels[train_start_idx:train_end_idx+1]
X_val,   y_val   = X_all[valid_start_idx:valid_end_idx+1],          labels[valid_start_idx:valid_end_idx+1]
X_test,  y_test  = X_all[test_start_idx:   ],                       labels[test_start_idx:   ]



# -------------------- Class weights打印 令训练集中数量最多的类别权重为 1。
# 其他类别的权重是按照“多少倍更稀少”给的。

cls, cnt = np.unique(y_train, return_counts=True)
max_cnt  = cnt.max()
weights  = np.asarray([max_cnt / dict(zip(cls, cnt))[c] for c in y_train], np.float32)
print("   class_weight:", {int(c): float(max_cnt/n) for c, n in zip(cls, cnt)})

# -------------------- LightGBM --------------------
train_ds = lgb.Dataset(X_train, y_train, weight=weights, free_raw_data=False)
val_ds   = lgb.Dataset(X_val,   y_val,                free_raw_data=False)

params = dict(
    objective        = "multiclass",
    num_class        = 3,
    seed             = SEED,
    learning_rate    = LR,
    num_leaves       = 127,
    max_depth        = -1,
    feature_fraction = 0.9,
    bagging_fraction = 0.9,
    bagging_freq     = 5,
    metric           = ["multi_logloss", "multi_error"],
    min_gain_to_split= 1e-3,
)

print("▶ Training LightGBM …")
model = lgb.train(
    params,
    train_ds,
    num_boost_round = N_ESTIM,
    valid_sets      = [train_ds, val_ds],
    valid_names     = ["train", "val"],
    callbacks       = [
        lgb.log_evaluation(200),
        lgb.early_stopping(EARLY_STOP)
    ]
)

# -------------------- Evaluation --------------------
y_pred = model.predict(X_test, num_iteration=model.best_iteration).argmax(1)
acc    = accuracy_score(y_test, y_pred)
print(f"\n🟢  Test Accuracy = {acc:.4f}\n")
print(classification_report(y_test, y_pred, digits=4))

# --------------------  Save --------------------
model_path = "/content/lgb_ic00_win1000_hor1000.txt"
model.save_model(model_path)
print("model saved to", model_path)


▶ Mounting Google Drive …
Mounted at /content/drive
 阅读中 …
   rows: 12060324   depth: 5
   labels computed: 48236
▶ Extracting features …


100%|██████████| 48236/48236 [00:08<00:00, 5433.36it/s]


   X: (48236, 66)  y: (48236,)
   class_weight: {0: 13.43525179856115, 1: 1.0, 2: 13.603804127883448}
▶ Training LightGBM …
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007942 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12060
[LightGBM] [Info] Number of data points in the train set: 38588, number of used features: 66
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 200 rounds
[200]	train's multi_logloss: 0.224265	train's multi_error: 0.0138232	val's multi_logloss: 1.0214	val's multi_error: 0.463912
[400]	train's multi_logloss: 0.0705171	train's multi_error: 0.000416481	val's multi_logloss: 0.997873	val's multi_error: 0.394613
Early stopping, best iteration is:
[299]	train's multi_logloss: 0.124862	train's multi_error: 0.00348059	va